# $ \text{Video Killed The Radio Star}$ $\color{red}{...Diffusion}$

Notebook by David Marx ([@DigThatData](https://twitter.com/digthatdata))

Shared under MIT license


# $\text{FAQ}$

**What is this?**

Point this notebook at a youtube url and it'll make a music video for you.

**How does this animation technique work?**

For each text prompt you provide, the notebook will...

1. Generate an image based on that text prompt (using stable diffusion)
2. Use the generated image as the `init_image` to recombine with the text prompt to generate variations similar to the first image. This produces a sequence of extremely similar images based on the original text prompt
3. Images are then intelligently reordered to find the smoothest animation sequence of those frames
3. This image sequence is then repeated to pad out the animation duration as needed

The technique demonstrated in this notebook was inspired by a [video](https://www.youtube.com/watch?v=WJaxFbdjm8c) created by Ben Gillin.

**How are lyrics transcribed?**

This notebook uses openai's recently released 'whisper' model for performing automatic speech recognition. 
OpenAI was kind enough to offer several different sizes of this model which each have their own pros and cons. 
This notebook uses the largest whisper model for transcribing the actual lyrics. Additionally, we use the 
smallest model for performing the lyric segmentation. Neither of these models is perfect, but the results 
so far seem pretty decent.

The first draft of this notebook relied on subtitles from youtube videos to determine timing, which was
then aligned with user-provided lyrics. Youtube's automated captions are powerful and I'll update the
notebook shortly to leverage those again, but for the time being we're just using whisper for everything
and not referencing user-provided captions at all.

**Something didn't work quite right in the transcription process. How do fix the timing or the actual lyrics?**

The notebook is divided into several steps. Between each step, a "storyboard" file is updated. If you want to
make modifications, you can edit this file directly and those edits should be reflected when you next load the
file. Depending on what you changed and what step you run next, your changes may be ignored or even overwritten.
Still playing with different solutions here.

**Can I provide my own images to 'bring to life' and associate with certain lyrics/sequences?**

Yes, you can! As described above: you just need to modify the storyboard. Will describe this functionality in
greater detail after the implementation stabilizes a bit more.

**This gave me an idea and I'd like to use just a part of your process here. What's the best way to reuse just some of the machinery you've developed here?**

Most of the functionality in this notebook has been offloaded to library I published to pypi called `vktrs`. I strongly encourage you to import anything you need 
from there rather than cutting and pasting function into a notebook. Similarly, if you have ideas for improvements, please don't hesitate to submit a PR!

**How can I support your work or work like it?**

This notebook was made possible thanks to ongoing support from [stability.ai](https://stability.ai/). The best way to support my work is to share it with your friends, [report bugs](https://github.com/dmarx/video-killed-the-radio-star/issues/new), [suggest features](https://github.com/dmarx/video-killed-the-radio-star/discussions) or to donate to open source non-profits :) 

## $0.$ Setup

In [ ]:
# @title # 📊 Check GPU Status

try:
    from vktrs.utils import gpu_info
except:
    import pandas as pd
    import subprocess
    
    def gpu_info():
        outv = subprocess.run([
            'nvidia-smi',
                # these lines concatenate into a single query string
                '--query-gpu='
                'timestamp,'
                'name,'
                'utilization.gpu,'
                'utilization.memory,'
                'memory.used,'
                'memory.free,'
                ,
            '--format=csv'
            ],
            stdout=subprocess.PIPE).stdout.decode('utf-8')

        header, rec = outv.split('\n')[:-1]
        return pd.DataFrame({' '.join(k.strip().split('.')).capitalize():v for k,v in zip(header.split(','), rec.split(','))}, index=[0]).T

gpu_info()

In [1]:
%%capture

# @title # 🛠️ Installations
!pip install vktrs[api,hf]

!pip install git+https://github.com/openai/whisper

# these are only needed for hf
!pip install "ipywidgets>=7,<8"
!sudo apt -qq install git-lfs
!git config --global credential.helper store

!pip install panel

In [ ]:
# @title # 🔑 Provide your API Key
# @markdown Running this cell will prompt you to enter your API Key below. 

# @markdown To get your API key, visit https://beta.dreamstudio.ai/membership

# @markdown ---

# @markdown A note on security best practices: **don't publish your API key.**

# @markdown We're using a form field designed for sensitive data like passwords.
# @markdown This notebook does not save your API key in the notebook itself,
# @markdown but instead loads your API Key into the colab environment. This way,
# @markdown you can make changes to this notebook and share it without concern
# @markdown that you might accidentally share your API Key. 
# @markdown 

use_stability_api = False # @param {type:'boolean'}
mount_gdrive = True # @param {type:'boolean'}

import os
from pathlib import Path
import time

from omegaconf import OmegaConf


os.environ['XDG_CACHE_HOME'] = os.environ.get(
    'XDG_CACHE_HOME',
    str(Path('~/.cache').expanduser())
)
if mount_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    Path('/content/drive/MyDrive/AI/models/.cache/').mkdir(parents=True, exist_ok=True) 
    # This rm+ln solution is not great. Be careful not to run this locally. 
    # Low risk, but could be annoying    
    !rm -rf /root/.cache
    !ln -sf /content/drive/MyDrive/AI/models/.cache/ /root/
    # Following line will be sufficient pending merge of https://github.com/openai/whisper/pull/257
    os.environ['XDG_CACHE_HOME']='/content/drive/MyDrive/AI/models/.cache'

model_dir_str=str(Path(os.environ['XDG_CACHE_HOME']))
proj_root_str = '${active_project}'
if mount_gdrive:
    proj_root_str = '/content/drive/MyDrive/AI/VideoKilledTheRadioStar/${active_project}'


# notebook config
cfg = OmegaConf.create({
    'active_project':str(time.time()),
    'project_root':proj_root_str,
    'gdrive_mounted':mount_gdrive,
    'use_stability_api':use_stability_api,
    'model_dir':model_dir_str,
    'output_dir':'${active_project}/frames'
})

with open('config.yaml','w') as fp:
    OmegaConf.save(config=cfg, f=fp.name)

###################

if use_stability_api:
    import os, getpass
    os.environ['STABILITY_KEY'] = getpass.getpass('Enter your API Key')
else:
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except ImportError:
        # assume local use
        pass
    
    from huggingface_hub import notebook_login

    # to do: if gdrive mounted, check for API token... somewhere on drive?
    # looks like we should be able to find the token through an environment variable
    notebook_login()


## $1.$ 📋 Create New Project (or set name to resume)

In [5]:
# @markdown To create a new project, enter a unique project name.
# @markdown If you leave `project_name` blank, the current unix timestamp will be used
# @markdown  (seconds since 1970-01-01 00:00).

# @markdown If you use the name of an existing project, the workspace will switch to that project.

# @markdown Non-alphanumeric characters (excluding '-' and '_') will be replaced with hyphens.

import time
from vktrs.utils import sanitize_folder_name
from omegaconf import OmegaConf

project_name = '' # @param {type:'string'}
if not project_name:
    project_name = str(time.time())

project_name = sanitize_folder_name(project_name)

workspace = OmegaConf.load('config.yaml')
workspace.active_project = project_name

with open('config.yaml','w') as fp:
    OmegaConf.save(config=workspace, f=fp.name)

## $2.$ 🔊 Infer speech from audio

In [ ]:
from omegaconf import OmegaConf
from pathlib import Path

workspace = OmegaConf.load('config.yaml')
use_stability_api = workspace.use_stability_api
model_dir = workspace.model_dir

root = workspace.project_root
root = Path(root)
root.mkdir(parents=True, exist_ok=True)


import copy
import datetime as dt
import gc
from itertools import chain, cycle
import json
import os
import re
import string
from subprocess import Popen, PIPE
import textwrap
import time
import warnings

from IPython.display import display
import numpy as np
import pandas as pd
import panel as pn
from tqdm.autonotebook import tqdm

import tokenizations
import webvtt
import whisper

from vktrs.utils import remove_punctuation
from vktrs.utils import get_audio_duration_seconds
from vktrs.youtube import (
    YoutubeHelper,
    parse_timestamp,
    vtt_to_token_timestamps,
    srv2_to_token_timestamps,
)

storyboard = OmegaConf.create()

d_ = dict(
    # all this does is make it so each of the following lines can be preceded with a comma
    # otw the first parameter would be offset from the other in the colab form
    _=""

    , video_url = 'https://www.youtube.com/watch?v=REojIUxX4rw' # @param {type:'string'}
    , audio_fpath = '' # @param {type:'string'}
    , whisper_seg = True # @param {type:'boolean'}
)
d_.pop('_')
storyboard.params = d_

if not storyboard.params.audio_fpath:
    storyboard.params.audio_fpath = None


# @markdown `video_url` - URL of a youtube video to download as a source for audio and potentially for text transcription as well.

# @markdown `audio_fpath` - Optionally provide an audio file instead of relying on a youtube download. Name it something other than 'audio.mp3', 
# @markdown                 otherwise it might get overwritten accidentally.

# @markdown `whisper_seg` - Whether or not to use openai's whisper model for lyric segmentation. This is currently the only option, but that will change in a few days.


storyboard_fname = root / 'storyboard.yaml'
with open(storyboard_fname,'wb') as fp:
    OmegaConf.save(config=storyboard, f=fp.name)


###############################
# Download audio from youtube #
###############################

video_url = storyboard.params.video_url

if video_url:
    # check if user provided an audio filepath (or we already have one from youtube) before attempting to download
    if storyboard.params.get('audio_fpath') is None:
        helper = YoutubeHelper(
            video_url,
            ydl_opts = {
                'outtmpl':{'default':str( root / f"ytdlp_content.%(ext)s" )},
                'writeautomaticsub':True,
                'subtitlesformat':'srv2/vtt'
                },
        )

        # estimate video end
        video_duration = dt.timedelta(seconds=helper.info['duration'])
        storyboard.params['video_duration'] = video_duration.total_seconds()

        audio_fpath = str( root / 'audio.mp3' )
        input_audio = helper.info['requested_downloads'][-1]['filepath']
        !ffmpeg -y -i "{input_audio}" -acodec libmp3lame {audio_fpath}

        # to do: write audio and subtitle paths/meta to storyboard
        storyboard.params.audio_fpath = audio_fpath

        if False:
            subtitle_format = helper.info['requested_subtitles']['en']['ext']
            subtitle_fpath = helper.info['requested_subtitles']['en']['filepath']

            if subtitle_format == 'srv2':
                with open(subtitle_fpath, 'r') as f:
                    srv2_xml = f.read() 
                token_start_times = srv2_to_token_timestamps(srv2_xml)
                # to do: handle timedeltas...
                #storyboard.params.token_start_times = token_start_times

            elif subtitle_format == 'vtt':
                captions = webvtt.read(subtitle_fpath)
                token_start_times = vtt_to_token_timestamps(captions)
                # to do: handle timedeltas...
                #storyboard.params.token_start_times = token_start_times

            # If unable to download supported subtitles, force use whisper
            else:
                storyboard.params.whisper_seg = True


# estimate video end
if storyboard.params.get('video_duration') is None:
    # estimate duration from audio file
    audio_fpath = storyboard.params['audio_fpath']
    storyboard.params['video_duration'] = get_audio_duration_seconds(audio_fpath)

if storyboard.params.get('video_duration') is None:
    raise RuntimeError('unable to determine audio duration. was a video url or path to a file supplied?')

# force use
storyboard.params.whisper_seg = True

with open(storyboard_fname,'wb') as fp:
    OmegaConf.save(config=storyboard, f=fp.name)

whisper_seg = storyboard.params.whisper_seg

###################################################
# 💬 Transcribe and segment speech using whisper #
###################################################

# handle OOM... or try to, anyway
if 'hf_helper' in locals():
    del hf_helper.img2img
    del hf_helper.text2img
    del hf_helper


if whisper_seg:
    from vktrs.asr import (
        #whisper_lyrics,
        #whisper_transcribe,
        #whisper_align,
        whisper_transmit_meta_across_alignment,
        whisper_segment_transcription,
    )

    #prompt_starts = whisper_lyrics(audio_fpath=storyboard.params.audio_fpath)

    audio_fpath = storyboard.params.audio_fpath
    #whispers = whisper_transcribe(audio_fpath)

    # to do: dropdown selectors
    segmentation_model = 'tiny'
    transcription_model = 'large'

    storyboard.params.whisper = dict(
        segmentation_model = segmentation_model
        ,transcription_model = transcription_model
    )

    whispers = {
        #'tiny':None, # 5.83 s
        #'large':None # 3.73 s
    }
    # accelerated runtime required for whisper
    # to do: pypi package for whisper

    # to do: use transcripts we've already built if we have them
    #scripts = storyboard.params.whisper.get('transcriptions')
    
    for k in set([segmentation_model, transcription_model]):
        #if k in scripts:

        options = whisper.DecodingOptions(
            language='en',
        )
        # to do: be more proactive about cleaning up these models when we're done with them
        model = whisper.load_model(k).to('cuda')
        start = time.time()
        print(f"Transcribing audio with whisper-{k}")
        
        # to do: calling transcribe like this unnecessarily re-processes audio each time.
        whispers[k] = model.transcribe(audio_fpath) # re-processes audio each time, ~10s overhead?
        print(f"elapsed: {time.time()-start}")
        del model
        gc.collect()
    
    #######################
    # save transcriptions #
    #######################

    transcriptions = {}
    transcription_root = root / 'whispers'
    transcription_root.mkdir(parents=True, exist_ok=True)
    for k in whispers:
        outpath = str( transcription_root / f"{k}.vtt" )
        transcriptions[k] = outpath
        with open(outpath,'w') as f:
            # to do: upstream PR to control verbosity
            whisper.utils.write_vtt(
                whispers[k]["segments"], # ...really?
                file=f
            )
    storyboard.params.whisper.transcriptions = transcriptions

    #tiny2large, large2tiny, whispers_tokens = whisper_align(whispers)
    # sanitize and tokenize
    whispers_tokens = {}
    for k in whispers:
        whispers_tokens[k] = [
        remove_punctuation(tok) for tok in whispers[k]['text'].split()
        ]

    # align sequences
    tiny2large, large2tiny = tokenizations.get_alignments(
        whispers_tokens[segmentation_model], #whispers_tokens['tiny'],
        whispers_tokens[transcription_model] #whispers_tokens['large']
    )
    #return tiny2large, large2tiny, whispers_tokens

    token_large_index_segmentations = whisper_transmit_meta_across_alignment(
        whispers,
        large2tiny,
        whispers_tokens,
    )
    prompt_starts = whisper_segment_transcription(
        token_large_index_segmentations,
    )


    ### checkpoint the processing work we've done to this point

    prompt_starts_copy = copy.deepcopy(prompt_starts)
    
    # to do: deal with timedeltas in asr.py and yt.py
    for rec in prompt_starts_copy:
        for k,v in list(rec.items()):
            if isinstance(v, dt.timedelta):
                rec[k] = v.total_seconds()
    
    storyboard.prompt_starts = prompt_starts_copy

    with open(storyboard_fname) as fp:
        OmegaConf.save(config=storyboard, f=fp.name)

###############################
# Review/Modify transcription #
###############################

# https://panel.holoviz.org/reference/widgets/Tabulator.html
pn.extension('tabulator') # I don't know that specifying 'tabulator' here is even necessary...

tabulator_formatters = {
    'bool': {'type': 'tickCross'}
}

df = pd.DataFrame(prompt_starts).rename(
    columns={
        'ts':'Timestamp (sec)',
        'prompt':'Lyric',
    }
)

if 'td' in df:
    del df['td']

df_pre = copy.deepcopy(df)
pn.widgets.Tabulator(df, formatters=tabulator_formatters)


## $3.$ 🎬 Animate

In [ ]:
import copy
import datetime as dt
from pathlib import Path
import random
import string

from bokeh.models.widgets.tables import (
    NumberFormatter, 
    BooleanFormatter,
    CheckboxEditor,
)
import numpy as np
from omegaconf import OmegaConf, DictConfig
import pandas as pd
import panel as pn
import PIL
from tqdm.autonotebook import tqdm

from vktrs.tsp import (
    tsp_permute_frames,
    batched_tsp_permute_frames,
)

from vktrs.utils import (
    add_caption2image,
    save_frame,
    remove_punctuation,
    get_image_sequence,
    archive_images,
)

# to do: is there a way to check if this is in the env already?
pn.extension('tabulator')

# this processes optional edits to the transcription (above) 
if ('prompt_starts' in locals()) \
and ('df_pre' in locals()):
    if isinstance(prompt_starts, DictConfig):
        prompt_starts = OmegaConf.to_container(prompt_starts)
    # update prompt_starts if any changes were made above
    if not np.all(df_pre.values == df.values):
        df_pre = copy.deepcopy(df)
        for i, rec in enumerate(prompt_starts):
            rec['ts'] = float(df.loc[i,'Timestamp (sec)'])
            rec['prompt'] = df.loc[i,'Lyric']
        
        # ...actually, I think the above code might not do anything
        # probably need to checkpoint prompt_starts into the storyboard on disk.
        # let's do that here just to be safe.    
        workspace = OmegaConf.load('config.yaml')
        root = Path(workspace.project_root)

        storyboard_fname = root / 'storyboard.yaml'
        storyboard = OmegaConf.load(storyboard_fname)

        storyboard.prompt_starts = prompt_starts
        with open(storyboard_fname) as fp:
            OmegaConf.save(config=storyboard, f=fp.name)


#####################################
# @title ## 🎨 Generate init images
#####################################

workspace = OmegaConf.load('config.yaml')
root = Path(workspace.project_root)

storyboard_fname = root / 'storyboard.yaml'
storyboard = OmegaConf.load(storyboard_fname)

prompt_starts = storyboard.prompt_starts
use_stability_api = workspace.use_stability_api
model_dir = workspace.model_dir

if use_stability_api:
    from vktrs.api import get_image_for_prompt
elif 'hf_helper' not in locals():
    from vktrs.hf import HfHelper
    # this needs to not be in the same cell as the login.
    # some sort of stupid race condition.
    try:
        hf_helper = HfHelper(
            download=False,
            model_path=str(Path(model_dir) / 'huggingface' / 'diffusers')
        )
    except:
        hf_helper = HfHelper(
            download=True,
            model_path=str(Path(model_dir) / 'huggingface' / 'diffusers')
        )

    # I give up.
    def get_image_for_prompt(*args, **kargs):
        result = hf_helper.get_image_for_prompt(*args, **kargs)
        return result.images


def get_variations_w_init(prompt, init_image, **kargs):
    return list(get_image_for_prompt(prompt=prompt, init_image=init_image, **kargs))

def get_close_variations_from_prompt(prompt, n_variations=2, image_consistency=.7):
    """
    prompt: a text prompt
    n_variations: total number of images to return
    image_consistency: float in [0,1], controls similarity between images generated by the prompt.
                        you can think of this as controlling how much "visual vibration" there will be.
                        - 0=regenerate each iandely identical
    """
    images = list(get_image_for_prompt(prompt))
    for _ in range(n_variations - 1):
        img = get_variations_w_init(prompt, images[0], start_schedule=(1-image_consistency))[0]
        images.append(img)
    return images


d_ = dict(
    _=''
    , theme_prompt = "deepdream, mural designed by a local artist" # @param {type:'string'}
    , height = 512 # @param {type:'integer'}
    , width = 512 # @param {type:'integer'}
    , display_frames_as_we_get_them = True # @param {type:'boolean'}
)
d_.pop('_')

regenerate_all_init_images = False # @param {type:'boolean'}

prompt_lag = False # @param {type:'boolean'}

# @markdown `theme_prompt` - Text that will be appended to the end of each lyric, useful for e.g. applying a consistent aesthetic style

# @markdown `display_frames_as_we_get_them` - Displaying frames will make the notebook slightly slower

# regenerate all images if the theme prompt has changed or user specifies

# @markdown `prompt_lag` - Extend prompt with lyrics from previous frame. Can improve temporal consistency of narrative. 
# @markdown  Especially useful for lyrics segmented into short prompts.

if d_['theme_prompt'] != storyboard.params.get('theme_prompt'):
    regenerate_all_init_images = True

storyboard.params.update(d_)

if regenerate_all_init_images:
    for i, rec in enumerate(prompt_starts):
        rec['frame0_fpath'] = None
        archive_images(i, root=root)
    print("archival process complete")

# anchor images will be regenerated if there's no associated frame0_fpath
# regenerate specific images if
# * manually tagged by user in df_regen
# * associated fpath doesn't exist (i.e. deleted)
if 'df_regen' in locals():
    for i, _ in df_regen.iterrows():
        rec = prompt_starts[i]
        regen = not _['keep']
        if rec.get('frame0_fpath') is None:
            regen = True
        elif not Path(rec['frame0_fpath']).exists():
            regen=True
        if regen:
            rec['frame0_fpath'] = None
            archive_images(i, root=root)
    print("archival process complete")


theme_prompt = storyboard.params.theme_prompt
display_frames_as_we_get_them = storyboard.params.display_frames_as_we_get_them
height = storyboard.params.height
width = storyboard.params.width

proj_name = workspace.active_project

print("Ensuring each prompt has an associated image")
for idx, rec in enumerate(prompt_starts):
    print(
        f"[{idx} | {rec['ts']}] - {rec['prompt']}"
    )
    lyric = rec['prompt']
    prompt = f"{lyric}, {theme_prompt}"
    if prompt_lag and (idx > 0):
        rec_prev = prompt_starts[idx -1]
        prompt = f"{rec_prev['prompt']}, {prompt}"
    if rec.get('frame0_fpath') is None:
        init_image = list(get_image_for_prompt(
              prompt,
              height=height,
              width=width,
              )
          )[0]
    # this shouldn't be necessary, but is a consequence of
    # the globbing thing we're doing atm
    if 'anchor' not in str(rec.get('frame0_fpath')):
        rec['frame0_fpath'] = save_frame(
            init_image,
            idx,
            root_path = root / 'frames',
            name='anchor',
            )

        if display_frames_as_we_get_them:
            print(lyric)
            display(init_image)


##############
# checkpoint #
##############

prompt_starts_copy = copy.deepcopy(prompt_starts)
storyboard.prompt_starts = prompt_starts_copy

with open(storyboard_fname) as fp:
    OmegaConf.save(config=storyboard, f=fp.name)


###############
# flag regens #
###############

df_regen = pd.DataFrame(prompt_starts)[['ts','prompt']].rename(
    columns={
        'ts':'Timestamp (sec)',
        'prompt':'Lyric',
    }
)

df_regen['keep'] = True

# move the "keep" column to the front
df_regen= df_regen[['keep', 'Timestamp (sec)', 'Lyric']]

pn.widgets.Tabulator(
    df_regen,
    formatters={'bool': BooleanFormatter()},
    editors={'bool':CheckboxEditor()}
    )


In [ ]:

# @title ## 🚀 Generate animation frames

###################
# improved resume #
###################

import copy
import datetime as dt
from itertools import cycle
from pathlib import Path

from omegaconf import OmegaConf
from PIL import Image
from vktrs.utils import (
    add_caption2image,
    get_image_sequence,
)


workspace = OmegaConf.load('config.yaml')
root = Path(workspace.project_root)

storyboard_fname = root / 'storyboard.yaml'
storyboard = OmegaConf.load(storyboard_fname)

if not 'prompt_starts' in locals():
    prompt_starts = OmegaConf.to_container(storyboard.prompt_starts)
else:
    ##########################
    # checkpoint any changes #
    ##########################
    prompt_starts_copy = copy.deepcopy(prompt_starts)

    storyboard.prompt_starts = prompt_starts_copy

    with open(storyboard_fname) as fp:
        OmegaConf.save(config=storyboard, f=fp.name)


#################################################
# Math                                          #
#                                               #
#    This block computes how many frames are    #
#    needed for each segment based on the start #
#    times for each prompt                      #
#################################################

# to do: 
# * make this more portable and add to vktrs lib

fps = 12 # @param {type:'integer'}
storyboard.params.fps = fps

ifps = 1/fps

# estimate video end
video_duration = storyboard.params['video_duration']

# dummy prompt for last scene duration
prompt_starts = OmegaConf.to_container(storyboard.prompt_starts)
prompt_starts.append({'ts':video_duration})

# make sure we respect the duration of the previous phrase
frame_start=0
prompt_starts[0]['anim_start']=frame_start
for i, rec in enumerate(prompt_starts[1:], start=1):
    rec_prev = prompt_starts[i-1]
    k=0
    while (rec_prev['anim_start'] + k*ifps) < rec['ts']:
        k+=1
    k-=1
    rec_prev['frames'] = k
    rec_prev['anim_duration'] = k*ifps
    frame_start+=k*ifps
    rec['anim_start']=frame_start

# drop the dummy frame
prompt_starts = prompt_starts[:-1]

# to do: given a 0 duration prompt, assume its duration is captured in the next prompt 
#        and guesstimate a corrected prompt start time and duration 


##############
# checkpoint #
##############

prompt_starts_copy = copy.deepcopy(prompt_starts)

storyboard.prompt_starts = prompt_starts_copy

with open(storyboard_fname) as fp:
    OmegaConf.save(config=storyboard, f=fp.name)


##################################
# Generate animation frames #
##################################

d_ = dict(
    _=''
    , n_variations=5 # @param {type:'integer'}
    , image_consistency=0.8 # @param {type:"slider", min:0, max:1, step:0.01}  
    , max_video_duration_in_seconds = 300 # @param {type:'integer'}
)
d_.pop('_')


# @markdown `fps` - Frames-per-second of generated animations

# @markdown `n_variations` - How many unique variations to generate for a given text prompt. This determines the frequency of the visual "pulsing" effect

# @markdown `image_consistency` - controls similarity between images generated by the prompt.
# @markdown - 0: ignore the init image
# @markdown - 1: true as possible to the init image

# @markdown `max_video_duration_in_seconds` - Early stopping if you don't want to generate a video the full duration of the provided audio. Default = 5min.


storyboard.params.update(d_)
storyboard.params.max_frames = storyboard.params.fps * storyboard.params.max_video_duration_in_seconds

# to do: compute and report unique of image generations

display_frames_as_we_get_them = storyboard.params.display_frames_as_we_get_them
image_consistency = storyboard.params.image_consistency
max_frames = storyboard.params.max_frames

n_variations = storyboard.params.n_variations
theme_prompt = storyboard.params.get('theme_prompt')


# load init_images and generate variations as needed
# to do: request multiple images in single request
print("Fetching variations")
for idx, rec in enumerate(prompt_starts):
    new_images = []
    images_fpaths = get_image_sequence(idx, root=root)
    curr_variation_count = len(images_fpaths)
    print(curr_variation_count)
    if curr_variation_count < n_variations:
        lyric = rec['prompt']
        prompt = f"{lyric}, {theme_prompt}"

        init_image = Image.open(rec['frame0_fpath'])
        # next line is here to permit user to specify more variations for a specific entry
        tot_variations = rec.get('n_variations', n_variations)
        tot_variations = min(tot_variations, rec['frames']) # don't generate variations we won't use
        tot_variations -= curr_variation_count  # only generate variations we still need
        for _ in range(tot_variations):
            img = get_variations_w_init(prompt, init_image, start_schedule=(1-image_consistency))[0]
            save_frame(
                img,
                idx,
                root_path= root / 'frames',
            )
            if display_frames_as_we_get_them:
                display(img)


##############
# checkpoint #
##############

prompt_starts_copy = copy.deepcopy(prompt_starts)

storyboard.prompt_starts = prompt_starts_copy

# to do: deal with these td objects
with open(storyboard_fname) as fp:
    OmegaConf.save(config=storyboard, f=fp.name)

In [ ]:
# @title ## 🎞️ Compile your video!

import shutil
from subprocess import Popen, PIPE

from omegaconf import OmegaConf
from tqdm.autonotebook import tqdm

from vktrs.tsp import (
    tsp_permute_frames,
    batched_tsp_permute_frames,
)

from vktrs.utils import (
    add_caption2image,
    get_image_sequence,
    save_frame,
    remove_punctuation,
)


# reload config
workspace = OmegaConf.load('config.yaml')
root = Path(workspace.project_root)

storyboard_fname = root / 'storyboard.yaml'
storyboard = OmegaConf.load(storyboard_fname)

########################
# rendering parameters #
########################

output_filename = 'output.mp4' # @param {type:'string'}
add_caption = False # @param {type:'boolean'}
optimal_ordering = True # @param {type:'boolean'}
upscale = False # @param {type:'boolean'}

# @markdown `add_caption` - Whether or not to overlay the prompt text on the image

# @markdown `optimal_ordering` - Intelligently permutes animation frames to provide a smoother animation.

# @markdown  `upscale`: Naively (lanczos interpolation) upscale video 2x. This can be a way to force
# @markdown  services like youtube to deliver your video without mangling it with compression
# @markdown  artifacts. Thanks [@gandamu_ml](https://twitter.com/gandamu_ml) for this trick!


# this parameter is currently not exposed in the form
max_variations_per_opt_pass = 15

if optimal_ordering:
    opt_batch_size = min(storyboard.params.n_variations, max_variations_per_opt_pass)

# I think it might be more efficient to write the video to the local disk first, then move it
# afterwards, rather than writing into google drive
final_output_filename = str( root / output_filename )
storyboard.params.output_filename = final_output_filename

# to do: move/duplicate fps computations here (?)
fps = storyboard.params.fps
input_audio = storyboard.params.audio_fpath

#####################################

# helper function for readability
def process_sequence(idx):
    im_paths = get_image_sequence(idx, root)
    images = [Image.open(fp) for fp in im_paths]
    
    if add_caption:
        images = [add_caption2image(im, rec['prompt']) for im in images]

    # to do: persist the ordering in the storyboard
    if optimal_ordering:
        images = batched_tsp_permute_frames(
            images,
            #max_variations_per_opt_pass
            opt_batch_size
        )
    return images

############################

cmd_in = ['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-']
cmd_out = ['-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '1', '-preset', 'veryslow', '-shortest', output_filename]

if input_audio:
  cmd_in += ['-i', str(input_audio)]

# NB: it might be more efficient to perform this upscaling step as a 
# separate step after compiling the video frames
if upscale:
    height=storyboard.params.height
    width=storyboard.params.width
    cmd_out = ['-vf', f'scale={2*width}x{2*height}:flags=lanczos'] + cmd_out


cmd = cmd_in + cmd_out

p = Popen(cmd, stdin=PIPE)
for idx, rec in enumerate(tqdm(storyboard.prompt_starts)): 
    frame_factory = cycle(process_sequence(idx))
    k = 0
    while k < rec['frames']:
        im = next(frame_factory)
        im.save(p.stdin, 'PNG')
        k+=1
p.stdin.close()

print("Encoding video...")
p.wait()

if output_filename != final_output_filename:
    print(f"Local video compilation complete. Moving video to: {final_output_filename}")
    shutil.move(output_filename, final_output_filename)
print("Video complete.")

In [ ]:
# @title ## 📺 Enjoy your animation!

# to do: Merge with 'compile' cell?

output_filename = storyboard.params.output_filename

download_video = True # @param {type:'boolean'}
compress_video = False # @param {type:'boolean'}

# @markdown Compressing to `*.tar.gz`` format can reduce filesize, which in turn reduces
# @markdown your download time. You may need to install additional software
# @markdown to "decompress" the file after downloading to view your video.

# @markdown NB: Your video will probably download way faster from https://drive.google.com

#  NB: only embed short videos
embed_video_in_notebook = False

if compress_video:
    uncompressed_fname = output_filename
    output_filename = f"{output_filename}.tar.gz"
    print(f"Compressing to: {output_filename}")
    !tar -czvf {output_filename} {uncompressed_fname}

if download_video:
    from google.colab import files
    files.download(output_filename)

if embed_video_in_notebook:
    from IPython.display import display, Video
    display(Video(output_filename, embed=True))

# ⚖️ I put on my robe and lawyer hat

### Notebook license

This notebook and the accompanying [git repository](https://github.com/dmarx/video-killed-the-radio-star/) and its contents are shared under the MIT license.

<!-- Note to self: lawyers should really be forced to use some sort of markup or pseudocode to eliminate ambiguity 

...oh shit, if laws were actually described in code, we could just run queries against it
-->

```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

### DreamStudio API TOS

The default behavior of this notebook uses the [DreamStudio](https://beta.dreamstudio.ai/) API to generate images. Users of the DreamStudio API are subject to the DreamStudio usage terms: https://beta.dreamstudio.ai/terms-of-service

### Stable Diffusion

As of the date of this writing (2022-09-29), all publicly available model checkpoints are subject to the restrictions of the Open RAIL license: https://huggingface.co/spaces/CompVis/stable-diffusion-license. 

